In [1]:
"""
Make pictures of a dataset using mapper
This file is an attempt to get one picture of ALL the data in a file
"""
memory = [[],[]]
Targetfile="toydata.txt"
#Weights disagreement as opposed to overlap
#Discrete processes could work too, but a more-continuous weighting is substantially faster due to 
#Technical considerations---numba only compiles linear algebra
Disagreement_weight = 100

import numpy as np

from tdamapper.learn import MapperAlgorithm
from tdamapper.cover import CubicalCover
from tdamapper.plot import MapperPlot

from sklearn.cluster import AgglomerativeClustering

from tdamapper._common import warn_user
from tdamapper.core import Cover, Proximity
from tdamapper.cover import BallCover
from numba import njit

#Read file
with open(Targetfile,"r") as file:
    line=file.readline()
    while line != "":
        tmp = line.split()
        memory[1].append(int(tmp[1]))
        memory[0].append(tmp[2])
        line=file.readline()

#Sort memory
memory[1],memory[0] = zip(*sorted(zip(memory[1], memory[0])))
memory[0] = list(memory[0])
memory[1] = list(memory[1])

# test that code works
print(memory)
import numpy as np
from numba import njit
from sklearn.cluster import AgglomerativeClustering
from tdamapper.cover import BallCover
from tdamapper.learn import MapperAlgorithm

#Define metrics
@njit(fastmath=True)
def hamming(x, y):
    return np.count_nonzero(x != y)

#Hamming distance with 9s as wildcards
@njit(fastmath=True)
def wild_hamming(x, y):
    return np.count_nonzero(np.logical_or(x != y, np.logical_or(x-y <= -5,x-y >=5)))

# Hamming distance but weighting disagreement on nonzero entries
# Differently than disagreement on zero entries
# Useful when padding matrices with zeroes to find differences without punishing overlaps
# EXPERIMENTAL: NOT GENERALLY A METRIC
#Disagreement_weight is by default 100
@njit(fastmath=True)
def assert_it_works_hamming(x,y):
    return (Disagreement_weight* np.count_nonzero(np.logical_and(np.logical_or(x != y, np.logical_or(x-y <= -5,x-y >=5)),x*y !=0))) + np.count_nonzero(x*y == 0) 


[['...'], [0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 3, 3, 4, 4, 4, 5, 5, 6, 8, 8, 8, 8, 9, 10, 10, 10, 10, 11, 12, 13, 13, 15, 15, 16, 19, 19, 20, 20, 22, 24, 24, 25, 27, 27, 28, 28, 29, 29, 29, 30, 30, 31, 31, 31, 32, 32, 32, 34, 34, 34, 35, 36, 36, 37, 38, 38, 38, 40, 40, 41, 41, 42, 43, 44, 45, 45, 46, 47, 48, 49, 50, 50, 50, 52, 52, 52, 54, 54, 54, 55, 57, 57, 57, 58, 58, 59, 60, 60, 60, 60, 61, 63, 64, 66, 66, 68, 68, 69, 69, 69, 69, 70, 71, 71, 72, 72, 73, 74, 76, 76, 76, 77, 77, 79, 79, 80, 81, 82, 85, 85, 86, 86, 86, 87, 87, 88, 89, 92, 94, 94, 95, 95, 97, 97, 98, 99, 99, 100, 101, 101, 101, 102, 103, 105, 105, 105, 106, 107, 107, 107, 107, 108, 108, 110, 110, 111, 112, 112, 113, 114, 114, 115, 116, 116, 117, 117, 117, 118, 118, 118, 119, 119, 120, 121, 121, 121, 122, 122, 122, 122, 122, 124, 125, 125, 125, 127, 127, 127, 128, 128, 129, 129, 131, 131, 132, 133, 134, 136, 136, 137, 137, 138, 138, 139, 139, 140, 141, 141, 142, 142, 144, 144, 145, 146, 147, 148, 149, 150, 150, 151, 151, 1

In [2]:
#Reindex to zero
the_amazing_subtracto = int( memory[1][0])
for a in range(len(memory[1])):
    memory[1][a] = int(memory[1][a]) - the_amazing_subtracto
print(memory[1])

#Translates a string to a numpy array of numbers
#For fast use with the 'hamming' and 'wild_hamming' metrics
transdict = {"A":1,"T":2,"C":3,"G":4,"N":9}
def str_to_np(index):
    out1 = [0 for i in range(int(memory[1][index]))]
    for a in memory[0][index]:
        out1.append(transdict[a])
    for i in range((int(memory[1][-1]) + int(len(memory[0][-1])) - int(memory[1][index]) - len(memory[0][index]))):
        out1.append(0)
    return np.array(out1)

#Draw a plot of the nerve of the strings containing the region of dna between l and width
def make_a_prety():
    X = np.array([str_to_np(x) for x in range(len(memory[0]))])
    y = X
    balls = BallCover(metric=assert_it_works_hamming, radius = 900)
    mapper_graph = MapperAlgorithm(cover=balls,clustering=AgglomerativeClustering(1)).fit_transform(X, y)
    colors = []
    for a in range(len(y)):
        colors.append(a)
    fig = MapperPlot(mapper_graph, dim=3, seed=42, iterations=100).plot_plotly(colors=colors,width=1200,height=800)
    fig.show(config={"scrollZoom": True})

#Draw a lot of plots of the (zero-tolerance) mapper quotients of the strings containing the region of dna between l and width
#for l in range(9,900,20 ):

make_a_prety()



[0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 3, 3, 4, 4, 4, 5, 5, 6, 8, 8, 8, 8, 9, 10, 10, 10, 10, 11, 12, 13, 13, 15, 15, 16, 19, 19, 20, 20, 22, 24, 24, 25, 27, 27, 28, 28, 29, 29, 29, 30, 30, 31, 31, 31, 32, 32, 32, 34, 34, 34, 35, 36, 36, 37, 38, 38, 38, 40, 40, 41, 41, 42, 43, 44, 45, 45, 46, 47, 48, 49, 50, 50, 50, 52, 52, 52, 54, 54, 54, 55, 57, 57, 57, 58, 58, 59, 60, 60, 60, 60, 61, 63, 64, 66, 66, 68, 68, 69, 69, 69, 69, 70, 71, 71, 72, 72, 73, 74, 76, 76, 76, 77, 77, 79, 79, 80, 81, 82, 85, 85, 86, 86, 86, 87, 87, 88, 89, 92, 94, 94, 95, 95, 97, 97, 98, 99, 99, 100, 101, 101, 101, 102, 103, 105, 105, 105, 106, 107, 107, 107, 107, 108, 108, 110, 110, 111, 112, 112, 113, 114, 114, 115, 116, 116, 117, 117, 117, 118, 118, 118, 119, 119, 120, 121, 121, 121, 122, 122, 122, 122, 122, 124, 125, 125, 125, 127, 127, 127, 128, 128, 129, 129, 131, 131, 132, 133, 134, 136, 136, 137, 137, 138, 138, 139, 139, 140, 141, 141, 142, 142, 144, 144, 145, 146, 147, 148, 149, 150, 150, 151, 151, 151, 153, 1